## 이 장에서 배울 내용

- 코루틴으로 작동하는 제너레이터의 동작과 상태
- 데커레이터를 이용치해서 코루틴을 자동으로 기동하기
- 제너레이터 객체의 close()와 throw() 메서드를 통해 호출자가 코루틴을 제어하는 방법
- 종료할 때 코루틴 값을 반환하는 방법
- 새로운 yield from 구문의 사용법과 의미
- 사용 예: 시뮬레이션의 동시 활동을 관리하기 위한 코루틴

# 1. 코루틴은 제너레이터에서 어떻게 진화했는가?


PEP380은 제너레이터 함수에 다음과 같이 두 가지 구문 변경을 정의해서 훨씬 더 유용하게 코루틴을 사용할 수 있도록 만들었다.

- 제너레이터가 값을 반환할 수 있다. 이전에는 제너레이터에서 return 문으로 값을 반환하면 SyntaxError가 발생했다.
- 기존 제너레이터가 하위 제너레이터에 위임하기 위해 수많은 판에 박힌 코드를 사용할 필요 없이, yield from 구문을 이용해서 복잡한 제너레이터를 더 작은 제너레이터로 리팩토링할 수 있게 한다.

# 2. 코루틴에서 사용되는 제너레이터의 기본 동작

In [9]:
# 가장 간단한 코루틴 사용 예
def simple_coroutine():
    print('-> 코루틴 시작')
    x = yield 
    print('-> 받은 코루틴 값: ' , x)
    

In [10]:
my_coro = simple_coroutine()
my_coro

<generator object simple_coroutine at 0x0000022B1DA58970>

In [11]:
next(my_coro)

-> 코루틴 시작


In [12]:
my_coro.send(51)

-> 받은 코루틴 값:  51


StopIteration: 

### 코루틴은 다음과 같은 4가지 상태 중 하나를 갖는다. inspect.getgeneratorstate() 함수를 이용해서 확인할 수 있다.  
   
  
- GEN_CREATED: 실행을 위해 대기하는 상태
- GEN_RUNNING: 인터프리터가 실행하고 있는 상태. 다중스레드 어플리케이션에서만 볼 수 있다.
- GEN_SUSPENDED: yield문에서 대기하고 있는 상태
- GEN_CLOSED: 실행이 완료된 상태

In [13]:
my_coro = simple_coroutine()
my_coro.send(523)

TypeError: can't send non-None value to a just-started generator

코루틴 객체를 생성하고 난 직후에 바로 None이 아닌 값을 전달하려고 하면 다음과 같은 오류가 발생한다.

In [15]:
#두 번 생성하는 코루틴
def simple_coro2(a):
    print('-> 시작 a :', a)
    b = yield a
    print('-> 받은 b = ', b)
    c = yield a + b
    print('-> 받은 c = ', c)

In [17]:
my_coro2 = simple_coro2(10)
from inspect import getgeneratorstate
getgeneratorstate(my_coro2)

'GEN_CREATED'

In [18]:
next(my_coro2)

-> 시작 a : 10


10

In [19]:
getgeneratorstate(my_coro2)

'GEN_SUSPENDED'

In [20]:
my_coro2.send(20)

-> 받은 b =  20


30

In [21]:
my_coro2.send(30)

-> 받은 c =  30


StopIteration: 

코루틴 실행은 yield 키워드에서 중단됨을 잘 알고 있어야 한다. 앞에서 설명한 것처럼 할당문에서는 실제 값을 할당하기 전에 = 오른쪽 코드를 실행한다. 즉,b = yeild a와 같은 코드에서는 나중에 호출자가 값을 보낸 후에야 변수 b가 설정된다. 이러한 방식에 익숙해지려면 신경을 더 써야 하지만, 이 방식을 제대로 알고 있어야 뒤에서 설명할 비동기 프로그래밍에서 yield의 용법을 이해할 수 있다.

# 3. 예제: 이동 평균을 계산하는 코루틴

In [22]:
# 이동 평균 코루틴 코드
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total / count

In [24]:
coro_avg = averager()
next(coro_avg)
coro_avg.send(10)

10.0

In [25]:
coro_avg.send(30)

20.0

In [26]:
coro_avg.send(5)

15.0

# 코루틴을 기동하기 위한 데커레이터

코루틴은 반드시 next(my_coro)를 호출해야하는데, 이를 편리하게 사용할 수 있도록 기동하는 데커레이터가 종종 사용된다. 대표적으로 @coroutine이 사용된다.

In [33]:
# 코루틴을 기동하기 위한 데커레이터
from functools import wraps

def coroutine(func):
    """데커레이터: 'func'를 기동해서 첫번째 yield까지 진행한다."""
    @wraps(func)
    def primer(*args, **kwargs):
        gen = func(*args, **kwargs)
        next(gen) # 제너레이터를 기동한다.
        return gen # 제너레이터를 반환한다.
    return primer

In [34]:
@coroutine
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total/count

In [36]:
coro_avg = averager()
from inspect import getgeneratorstate
getgeneratorstate(coro_avg)

'GEN_SUSPENDED'

In [37]:
coro_avg.send(10)

10.0

In [38]:
coro_avg.send(20)

15.0

In [39]:
coro_avg.send(15)

15.0

yield from 구문은 자동으로 자신을 실행한 코루틴을 기동시키므로 @coroutine 데커레이터와 함께 사용할 수 없다.  

이제 코루틴의 핵심 기능에 대해 자세히 살펴보자. 다음 절에서는 코루틴을 종료시키는 메서드와 코루틴에 예외를 던지는 메서드를 설명한다.

# 5. 코루틴 종료와 예외처리

코루틴 안에서 발생한 예외를 처리하지 않으면, next()나 send()로 코루틴을 호출한 호출자에 예외가 전파된다.

In [40]:
coro_avg = averager()
coro_avg.send(40)

40.0

In [41]:
coro_avg.send(50)

45.0

In [42]:
coro_avg.send('spam')

TypeError: unsupported operand type(s) for +=: 'float' and 'str'

In [43]:
coro_avg.send(60)

StopIteration: 

위 예제처럼 코루틴에 구분 표시를 전송해서 코루틴을 종료할 수 있다.

제너레이터 객체는 호출자가 코루틴에 명시적으로 예외를 전달할 수 있게 해주는 throw()와 close() 메서드를 제공한다.

generator.throw(exc_type[, exc_value[, traceback]]) : 제너레이터가 중단한 곳의 yield 표현식에 예외를 전달한다. 제너레이터가 예외를 처리하면 제어 흐름이 다음 yield문까지 진행하고, 생성된 값은 generator.throw() 호출 값이 된다. 제너레이터가 예외를 처리하지 않으면 호출자까지 예외가 전파된다.

generator.close() : 제너레이터가 실행을 중단한 yield 표현식이 GeneratorExit 예외를 발생시키게 만든다. 제너레이터가 예외를 처리하지 않거나 StopIteration 예외를 발생시키면 아무런 에러도 호출자에 전달되지 않는다.

In [44]:
class DemoException(Exception):
    """설명에 사용할 예외 유형"""

def demo_exc_handling():
    print('-> coroutine started')
    while True:
        try:
            x = yield
        except DemoException:
            print('*** DemoException handled. Continuing...')
        else: # 예외가 발생하지 않으면 받은 값을 출력한다.
            print('-> coroutine received: {!r}'.format(x))
    raise RuntimeError('This line should never run.') # 이 코드는 실행되지 않음
    # 무한루프는 처리되지 않은 예외에서만 중지될 수 있으며
    # 예외가 처리되지 않으면 코루틴의 실행이 바로 중지되기 때문

In [45]:
exc_coro = demo_exc_handling()
next(exc_coro)

-> coroutine started


In [46]:
exc_coro.send(11)

-> coroutine received: 11


In [47]:
exc_coro.throw(DemoException)

*** DemoException handled. Continuing...


In [48]:
getgeneratorstate(exc_coro)

'GEN_SUSPENDED'

In [49]:
# 처리되지 않는 예외를 더니면 코루틴이 중단된다.

exc_coro.throw(ZeroDivisionError)

ZeroDivisionError: 

In [50]:
getgeneratorstate(exc_coro)

'GEN_CLOSED'

코루틴이 어떻게 종료되든 어떤 정리 코드를 실행해야 하는 경우에는 try/finally 블록 안에 해당 코드를 넣어야 한다.

In [51]:
class DemoException(Exception):
    """설명에 사용할 예외 유형"""

def demo_finally():
    print('-> coroutine started')
    try:
        while True:
            try:
                x = yield
            except DemoException:
                print('*** DemoException handled. Continuing...')
            else: # 예외가 발생하지 않으면 받은 값을 출력한다.
                print('-> coroutine received: {!r}'.format(x))
    finally:
        print('-> coroutine ending')

In [52]:
exc_coro = demo_finally()
next(exc_coro)

-> coroutine started


In [53]:
exc_coro.send(11)

-> coroutine received: 11


In [54]:
exc_coro.throw(DemoException)

*** DemoException handled. Continuing...


In [55]:
exc_coro.throw(ZeroDivisionError) # 에러가 발생했지만 출력은 됨

-> coroutine ending


ZeroDivisionError: 

# 6. 코루틴에서 값 반환하기

의미 있는 값을 생성하지는 않지만 최후에 어떤 의미 있는 값을 반환하는 코루틴도 있음을 설명

In [56]:
from collections import namedtuple

Result = namedtuple('Result', 'count average')

def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break # 값을 반환하려면 코루틴이 정상적으로 종료되어야 한다.
            # 따라서 이 averager 버전에서는 루프를 빠져나오는 조건을 검사한다.
        total += term
        count += 1
        average = total/count
    return Result(count, average) # nametuple 반환

In [57]:
coro_avg = averager()
next(coro_avg)
coro_avg.send(10) # 값을 생성하지 않음

In [58]:
coro_avg.send(30)

In [59]:
coro_avg.send(6.5)

In [60]:
coro_avg.send(None) # None을 보내면 루프를 빠져나오고 코루틴이 결과를 반환하면서 종료
# 예외 객체의 value 속성에는 반환된 값이 들어있다.

StopIteration: Result(count=3, average=15.5)

In [62]:
# 코루틴이 반환한 값을 가져오는 방법 
coro_avg = averager()
next(coro_avg)
coro_avg.send(10) # 값을 생성하지 않음
coro_avg.send(30)
coro_avg.send(6.5)

In [63]:
try:
    coro_avg.send(None)
except StopIteration as exc:
    result = exc.value
result

Result(count=3, average=15.5)

# 7. yield from 사용하기

yield from은 완전히 새로운 언어 구성체이다. 제너레이터 gen()이 yield from subgen()을 호출하고, subgen()이 이어받아 값을 생성하고 gen()의 호출자에 반환한다.

14장에서 yield from을 for 루프 안의 yield에 대한 단축문으로 사용할 수 있다고 설명했다.

In [64]:
def gen():
    for c in 'AB':
        yield c
    for i in range(1,3):
        yield i


In [65]:
list(gen())

['A', 'B', 1, 2]

위 코드는 다음과 같이 바꿀 수 있다.

In [66]:
def gen():
    yield from 'AB'
    yield from range(1,3)
    
list(gen())

['A', 'B', 1, 2]

yield from x 표현식이 x 객체에 대해 첫 번째로 하는 일은 iter(x)를 호출해서 x의 반복자를 가져오는 것이다. 그러나 yield from의 주요한 특징은 가장 바깥쪽 호출자와 가장 안쪽에 있는 하위 제너레이터 사이에 양방향 채널을 열어준다는 것이다.

yield from을 사용하기 위한 주요 용어는 다음과 같다.

- 대표 제너레이터 : yield from <반복형> 표현식을 담고 있는 제너레이터 함수
- 하위 제너레이터 : yield from 표현식 중 <반복형>에서 가져오는 제너레이터
- 호출자 : 대표 제너레이터를 호출하는 코드

# 8. yield from의 의미

- 하위 제너레이터가 생성하는 값은 모두 대표 제너레이터의 호출자에 바로 전달
- send()를 통해 대표 제너레이터에 전달한 값은 모두 하위 제너레이터에 직접 전달된다. 값이 None이면 하위 제너레이터의 __next__() 메서드가 호출된다. None이 아니면 하위 제너레이터의 send() 메서드가 호출된다. 호출된 메서드에서 StopIteration 예외가 발생하면 대표 제너레이터의 실행이 재개된다. 그 외의 예외는 대표 제너레이터 전달된다.
- 제너레이터나 하위 제너레이터에서 return expr 문을 실행하면 제너레이터를 빠져나온 후 StopIteration(expr) 예외가 발생한다.
- 하위 제너레이터가 실행을 마친 후 발생한 StopIteration 예외의 첫 번째 인수가 yield from 표현식의 값이 된다.
- 대표 제너레이터에 던져진 GeneratorExit 이외의 예외는 하위 제너레이터의 throw() 메서드에 전달된다. throw() 메서드를 호출해서 StopIteration 예외가 발생하면 대표 제너레이터의 실행이 재개된다. 그 외의 예외는 대표 제너레이터에 전달된다.
- GeneratorExit 예외가 대표 제너레이터에 던져지거나 대표 제너레이터의 close() 메서드가 호출되면 하위 제너레이터의 close() 메서드가 호출된다. 그 결과 예외가 발생하면 발생한 예외가 대표 제너레이터에 전파된다. 그렇지 않으면 대표 제너레이터에서 GeneratorExit 예외가 발생한다.